# word2vecの挙動を確認する

In [1]:
from gensim.models import KeyedVectors

In [2]:
model = KeyedVectors.load_word2vec_format('model.vec', binary=True)

print('model loaded')

model loaded


In [3]:
from gensim.models.word2vec import Word2Vec

In [4]:
model['日当']

array([ -7.23709285e-01,   1.88759658e-02,   5.43161273e-01,
         1.43577307e-01,   5.70250213e-01,   2.43271410e-01,
         9.17903706e-02,   3.06424171e-01,  -3.41994941e-01,
        -2.85300761e-01,  -1.28893167e-01,   3.38842452e-01,
         2.50624299e-01,   1.11641787e-01,  -6.20470464e-01,
         4.70917434e-01,   3.32915157e-01,  -5.02526224e-01,
         2.24614680e-01,   3.84199679e-01,  -6.51292562e-01,
         3.48061711e-01,   4.75467712e-01,  -6.16732948e-02,
         6.05587810e-02,  -4.36736882e-01,  -3.72240067e-01,
         8.62543508e-02,  -3.75325859e-01,  -1.04761068e-02,
         7.46260166e-01,  -8.99853185e-02,   2.85969358e-02,
        -1.63807109e-01,   4.98178124e-01,   4.55942482e-01,
         6.25100955e-02,  -5.98474085e-01,   2.62162060e-01,
        -1.84283312e-02,   9.55360234e-02,  -5.75984538e-01,
        -6.33886874e-01,   4.59632039e-01,   4.45628613e-01,
         5.34642220e-01,   2.12091003e-02,  -7.12478995e-01,
         4.85079795e-01,

In [5]:
model['MYOPE_QA_ID:0']

KeyError: "word 'MYOPE_QA_ID:0' not in vocabulary"

ボキャブラリがない場合はエラーになる

In [6]:
from gensim.similarities.docsim import WmdSimilarity
import numpy as np

corpus = np.array([
    ['こんにちは', '交通', '費', '申請'],
    ['海外', '日当' 'MYOPE_QA_ID:888'],
    ['国内', '出張', '担当', '者', '教え', '下さい'],
    ['海外', '出張', '申請', 'ルート', '承認', '者', '変更', 'MYOPE_QA_ID:999']
])

In [7]:
wmd = WmdSimilarity(corpus, model, num_best=10)

In [8]:
wmd['MYOPE_QA_ID:999']

[(3, 0.42747335809607512),
 (0, 0.42612267237868551),
 (1, 0.41935011413942114),
 (2, 0.41906706063564897)]

In [9]:
wmd[['海外', '日当', 'MYOPE_QA_ID:0']]

[(1, 0.60361194695176656),
 (3, 0.47628312326919181),
 (2, 0.44740058255077148),
 (0, 0.44600532859279046)]

In [10]:
wmd[['海外', '日当']]

[(1, 0.60361194695176656),
 (3, 0.47628312326919181),
 (2, 0.44740058255077148),
 (0, 0.44600532859279046)]

In [11]:
wmd[['海外', '日当', 'MYOPE_QA_ID:888']]

[(1, 0.60361194695176656),
 (3, 0.47628312326919181),
 (2, 0.44740058255077148),
 (0, 0.44600532859279046)]

MYOPE_QA_IDを変えても同じスコアが出た

wmd自体には他にメソッドがないため確認できなかったが、MYOPE_QA_IDを無視されてそうな挙動

### wmdistanceをみてみる

build_vocabで単語を登録してみたい

In [13]:
import copy

w2v = Word2Vec()
print(w2v.wmdistance(['海外'], ['日当']))
w2v.wv = copy.deepcopy(model)
print(w2v.wmdistance(['海外'], ['日当']))
print(w2v.wmdistance(['MYOPE_QA_ID:888'], ['日当']))
print(w2v.wmdistance(['MYOPE_QA_ID:888'], ['MYOPE_QA_ID:999']))
sentences = [['海外', '出張', '申請', 'ルート', '承認', '者', '変更', 'MYOPE_QA_ID:999'], ['海外', '日当' 'MYOPE_QA_ID:888']]
w2v.scan_vocab(sentences)
w2v.build_vocab(sentences, update=True)
print(w2v.wmdistance(['ほげほげ', 'MYOPE_QA_ID:888'], ['日当']))

inf
1.3133870363235474
inf
inf


ValueError: all the input array dimensions except for the concatenation axis must match exactly

登録されていないvocabは値がないために距離がinf(無限)になる?

https://github.com/RaRe-Technologies/gensim/issues/1162

> 単語がない場合（おそらく、新しい素材が以前のトレーニングでは非常に小さいか、または非常に冗長であるため）、後続のトレーニングがモデルにとって実質的なメリットではない場合もあります。 （このインクリメンタル・ボキャブラリー拡張オプションは実験的な機能と見なされるのが最善であり、使用ごとに慎重に評価する必要があります。実際の「オンライン」トレーニング・オプションではありません。小さな/冗長な新しいバッチを使用してupdate-vocabを簡単に呼び出すプロセスには注意が必要です。

vocablaryを追加する方法、このエラーの解決法がわからない

In [14]:
w2v = Word2Vec(['テスト', '日当'], min_count=1)
w2v.scale_vocab(min_count=1)
w2v.build_vocab(sentences, update=True)

## これなら通る

In [15]:
w2v = Word2Vec(['テスト', '日当'], min_count=1)
w2v.wv = copy.deepcopy(model)
w2v.scale_vocab(min_count=1)
w2v.build_vocab(sentences, update=True)

## これは通らない

ValueError: negative dimensions are not allowed

イチから学習モデルを作らないとだめなのかもしれない。
その際にもちゃんと追加学習できるかを慎重に確認しながら学習モデルを作らないといけない?(1センテンスあたりの単語量とか、vocabのscaleとか)

In [16]:
model['日当']

array([ -1.27825245e-01,   3.33396997e-03,   9.59359333e-02,
         2.53593605e-02,   1.00720517e-01,   4.29678448e-02,
         1.62124857e-02,   5.41222095e-02,  -6.04049042e-02,
        -5.03912866e-02,  -2.27657724e-02,   5.98480925e-02,
         4.42665517e-02,   1.97187457e-02,  -1.09590679e-01,
         8.31758529e-02,   5.88011853e-02,  -8.87587592e-02,
         3.96725982e-02,   6.78593218e-02,  -1.15034640e-01,
         6.14764467e-02,   8.39795470e-02,  -1.08930543e-02,
         1.06962025e-02,  -7.71387070e-02,  -6.57469481e-02,
         1.52346864e-02,  -6.62919804e-02,  -1.85034378e-03,
         1.31808296e-01,  -1.58936698e-02,   5.05093765e-03,
        -2.89324522e-02,   8.79907757e-02,   8.05309042e-02,
         1.10408543e-02,  -1.05705567e-01,   4.63044085e-02,
        -3.25490651e-03,   1.68740638e-02,  -1.01733349e-01,
        -1.11960351e-01,   8.11825693e-02,   7.87092149e-02,
         9.44312513e-02,   3.74606019e-03,  -1.25841692e-01,
         8.56772810e-02,

reset_weights()を試してみた

In [17]:
w2v = Word2Vec(min_count=1)
w2v.wv = copy.deepcopy(model)
w2v.reset_weights()
w2v.build_vocab(sentences, update=True)

In [34]:
w2v.wv['MYOPE_QA_ID:888']

KeyError: "word 'MYOPE_QA_ID:888' not in vocabulary"

In [33]:
w2v.train(sentences, total_examples=len(sentences), epochs=w2v.iter)

5

In [32]:
w2v.wv['MYOPE_QA_ID:888']

KeyError: "word 'MYOPE_QA_ID:888' not in vocabulary"

build_vocabはできたが、ボキャブラリーは登録されない?

trainもしてみたがかわらず。


そもそもtrain関数を使うべきじゃない?

https://qiita.com/shinochin/items/661cbed78df3cb83b2aa

> train関数を使うとtraining rateのalphaを少しずつ下げていくわけですが、trainを複数回実行するということはalphaを複数回上げ下げするわけなので、適切なSGDにならないとのことでした。